# ETL Mensual - Procesamiento Incremental

Este notebook ejecuta el proceso ETL mensual para las fuentes de datos:
- **Inspecciones**: archivos `inspecciones_YYYY_MM.xlsx` en `data/raw/inspecciones/`
- **Consumo**: archivos `consumo_YYYY_MM.xlsx` en `data/raw/consumo/`

## Flujo:
1. Detecta meses pendientes (archivos en `raw/` que no tienen parquet en `interim/`)
2. Procesa cada mes: limpia, normaliza y guarda en `data/interim/<fuente>/year=YYYY/month=MM/`
3. Solo procesa meses nuevos (incremental) - salvo que se fuerce `overwrite=True`

In [1]:
import os
import sys
import pandas as pd
from tqdm import tqdm

# Configurar paths
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Activar autoreload para desarrollo
%load_ext autoreload
%autoreload 2

In [2]:
# Instalar dependencias si es necesario
try:
    import openpyxl
except ImportError:
    print("Instalando openpyxl...")
    %pip install openpyxl

from src.data import etl

## Configuración

Define los directorios base y las fuentes a procesar.

In [ ]:
# Directorios base
RAW_DIR = '../data/raw'
INTERIM_DIR = '../data/interim'

# Fuentes a procesar
SOURCES = ['inspecciones', 'consumo']

# Si True, reprocesa todo aunque ya exista (útil para backfill o cambios de reglas)
OVERWRITE = True

## Ejecutar ETL Mensual

Ejecuta el proceso ETL incremental. Solo procesa meses nuevos por defecto.

In [5]:
%%time

# Ejecutar ETL mensual
summary = etl.run_monthly_etl(
    raw_dir=RAW_DIR,
    interim_dir=INTERIM_DIR,
    sources=SOURCES,
    overwrite=OVERWRITE
)


[INSPECCIONES]
  [INFO] 41 meses pendientes
     [(2021, 3), (2021, 4), (2021, 5), (2021, 6), (2021, 7)] ... [(2024, 4), (2024, 5), (2024, 6), (2024, 7), (2024, 8)]


  Procesando inspecciones:   2%|▏         | 1/41 [00:00<00:07,  5.58it/s]

  [PROC] Procesando inspecciones 2021-03...
  [OK] Guardado: ../../data/interim\inspecciones\year=2021\month=03\inspecciones.parquet (248 registros)


  Procesando inspecciones:   5%|▍         | 2/41 [00:00<00:06,  6.31it/s]

  [PROC] Procesando inspecciones 2021-04...
  [OK] Guardado: ../../data/interim\inspecciones\year=2021\month=04\inspecciones.parquet (234 registros)
  [PROC] Procesando inspecciones 2021-05...


...

  [OK] Guardado: ../../data/interim\consumo\year=2024\month=12\consumo.parquet (12139 registros)
  [PROC] Procesando consumo 2025-01...


  Procesando consumo:  98%|█████████▊| 58/59 [05:54<00:06,  6.10s/it]

  [OK] Guardado: ../../data/interim\consumo\year=2025\month=01\consumo.parquet (12775 registros)
  [PROC] Procesando consumo 2025-02...


  Procesando consumo: 100%|██████████| 59/59 [06:00<00:00,  6.11s/it]

  [OK] Guardado: ../../data/interim\consumo\year=2025\month=02\consumo.parquet (12867 registros)
  [OK] consumo: 59 procesados, 0 saltados
CPU times: total: 3min 56s
Wall time: 6min 6s


## Resumen de Procesamiento

In [8]:
# Mostrar resumen
print("\n" + "="*60)
print("RESUMEN FINAL")
print("="*60)
for source, stats in summary.items():
    print(f"\n{source.upper()}:")
    print(f"  - Procesados: {stats['processed']}")
    print(f"  - Saltados: {stats['skipped']}")
    print(f"  - Total pendientes encontrados: {stats['total_pending']}")


RESUMEN FINAL

INSPECCIONES:
  - Procesados: 41
  - Saltados: 0
  - Total pendientes encontrados: 41

CONSUMO:
  - Procesados: 59
  - Saltados: 0
  - Total pendientes encontrados: 59


## Verificar Archivos Procesados

Lista los parquets generados en `interim/` para verificar.

In [9]:
import glob
import re

# Listar archivos procesados
for source in SOURCES:
    pattern = os.path.join(INTERIM_DIR, source, "year=*", "month=*", f"{source}.parquet")
    files = sorted(glob.glob(pattern))
    print(f"\n{source.upper()}: {len(files)} archivos procesados")
    if len(files) <= 10:
        for f in files:
            # Extraer year/month del path
            match = re.search(r"year=(\d{4})/month=(\d{2})", f.replace("\\", "/"))
            if match:
                print(f"  - {match.group(1)}-{match.group(2)}")
    else:
        print(f"  (mostrando primeros 5 y últimos 5)")
        for f in files[:5]:
            match = re.search(r"year=(\d{4})/month=(\d{2})", f.replace("\\", "/"))
            if match:
                print(f"  - {match.group(1)}-{match.group(2)}")
        print("  ...")
        for f in files[-5:]:
            match = re.search(r"year=(\d{4})/month=(\d{2})", f.replace("\\", "/"))
            if match:
                print(f"  - {match.group(1)}-{match.group(2)}")


INSPECCIONES: 44 archivos procesados
  (mostrando primeros 5 y últimos 5)
  - 2021-01
  - 2021-02
  - 2021-03
  - 2021-04
  - 2021-05
  ...
  - 2024-04
  - 2024-05
  - 2024-06
  - 2024-07
  - 2024-08

CONSUMO: 62 archivos procesados
  (mostrando primeros 5 y últimos 5)
  - 2020-01
  - 2020-02
  - 2020-03
  - 2020-04
  - 2020-05
  ...
  - 2024-10
  - 2024-11
  - 2024-12
  - 2025-01
  - 2025-02
